In [1]:
import numpy as np
import pandas as pd
import h5py

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

from glob import glob
#from os import listdir
#from os.path import splitext
import logging
import os

In [2]:
19636 + 19309

38945

In [3]:
dir_img = '/mnt/sda1/tiles_2018_wien/'

## finding files

In [4]:
ids_ortho = []
ids_ground_truth = []

sufix_ortho = 'ortho/'
sufix_gt = 'ground_truth/'

# loop over all files found in directory
for file in os.listdir(dir_img+sufix_ortho):
    # create path to ground truth
    path_gt = dir_img + sufix_gt + 'tile_ground_truth' + file[file.rfind('_'):]
    # if file is .tif and the ID is alo found in the grooound truth dictonary
    if file[file.rfind('.'):] == '.tif' and os.path.isfile(path_gt):
        # add path to lists
        ids_ortho.append(dir_img + sufix_ortho + file)
        ids_ground_truth.append(path_gt)

In [5]:
len(ids_ground_truth)

19303

In [30]:
print(ids_ortho[0])
print(ids_ortho[288])
print(ids_ortho[399])
print(ids_ortho[666])
print(ids_ortho[5000])
print(ids_ortho[6969])
print(ids_ortho[9990])

/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_10315.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_3638.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_8308.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_5838.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_10582.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_3371.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_3106.tif


In [6]:
print(ids_ortho[0])
print(ids_ortho[288])
print(ids_ortho[399])
print(ids_ortho[666])
print(ids_ortho[5000])
print(ids_ortho[6969])
print(ids_ortho[9990])

/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_10315.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_3638.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_8308.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_5838.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_10582.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_3371.tif
/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_3106.tif


In [7]:
from osgeo import gdalconst

def tif2array(input_file, dtype=np.uint8):
    """
    read GeoTiff and convert to numpy.ndarray.
    inputs:
        input_file (str) : the name of input GeoTiff file.
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive.
    """
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    
    if dataset is None:
        return None
    
    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=dtype)
    
    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)
        
    #image = image[2:-2,2:-2,:]
        
    return image

In [8]:
def cut_img(img, x, y):
    """
    cut input numpy array to the width(x) and height(y)
    inputs:
        img (np.array) : image as numpy array
        x (int) : target width
        y (int) : target height
    return:
        img (np.array) : image cutted to the target width(x) and height(y)
    """
    # set pixel sizes
    x_i, y_i, z_i = img.shape
    # dict to store the sliceing information
    d = {}
    
    for var, var_i, key in [(x, x_i, 'x'), (y, y_i, 'y')]:
        # if image pixel size is grater than the target pixel size
        if (var_i > var):
            # if even cut same amount of pixels from both sides
            if var_i%2 == 0:
                sub = int(var_i/2 - var/2)
                d[key+'0'] = sub
                d[key+'1'] = sub
            # if odd cut 1 pixel more from right/bottom
            else:
                sub = int(var_i/2 - var/2)
                d[key+'0'] = sub
                d[key+'1'] = sub + 1
        else:
            print('image too small')
    # cut image
    img = img[d['x0']:-d['x1'],d['y0']:-d['y1']]
    
    return img

In [9]:
# pass ids_ortho, ids_dsm, ids_dtm, ids_slope
def create_array(ids, dtype):
    """
    creates numpy array with all images stacked
    inputs:
        ids (list) : list of paths to image files
        dtype (dtype) : dtype for storing the loaded image
    return:
        arr (np.array) : numpy array containing all the images stacked
    """
    
    imgs = []
    
    if dtype == np.uint8:
        # add all
        for i in ids:
            # load image to numpy array
            img = tif2array(i, np.uint8)
            # cut into right shape
            img = cut_img(img, 512, 512)
            # append array to list
            imgs.append(img)
            
        # convert list with arrays to numpy array
        arr = np.stack(imgs, axis=0)
        print(arr.shape)
            
        # calculate mean
        #mean = np.mean(arr, dtype='float32')
        # calculate standard deviation
        #std = np.std(arr, dtype='float32')
    
    else:
        # add all
        for i in ids:
            
            # load image to numpy array
            img = tif2array(i, np.float32)
            # cut into right shape
            img = cut_img(img, 512, 512)
            # append array to list
            imgs.append(img)
            x, y, z = img.shape

        # convert list with arrays to numpy array
        arr = np.stack(imgs, axis=0)
        arr[arr < 0] = np.nan
        print(arr.shape)

        # calculate mean
        #mean = np.nanmean(arr, dtype='float32')
        # calculate standard deviation
        #std = np.nanstd(arr, dtype='float32')
        # convert nan to 0
        arr = np.nan_to_num(arr)
    
    # normalize data
    #arr = (arr - mean) / std
    
    return arr#, mean, std

In [31]:
arr_ortho = create_array(ids_ortho[15000:19303], np.uint8)
arr_ground_truth = create_array(ids_ground_truth[15000:19303], np.uint8)

(4303, 512, 512, 4)
(4303, 512, 512, 1)


In [32]:
# create mask
m = np.ma.make_mask(arr_ground_truth)

In [33]:
# apply mask
arr_ortho *= m

In [34]:
# create 256
arr_ortho_256 = np.concatenate([arr_ortho[:, :256, :256], \
arr_ortho[:, 256:, :256], \
arr_ortho[:, :256, 256:], \
arr_ortho[:, 256:, 256:]], axis=0)

arr_ground_truth_256 = np.concatenate([arr_ground_truth[:, :256, :256], \
arr_ground_truth[:, 256:, :256], \
arr_ground_truth[:, :256, 256:], \
arr_ground_truth[:, 256:, 256:]], axis=0)

In [35]:
del arr_ortho

In [36]:
del arr_ground_truth

In [37]:
arr_ortho_256.shape

(17212, 256, 256, 4)

In [38]:
arr_ground_truth_256.shape

(17212, 256, 256, 1)

In [18]:
## delete empty tiles

In [39]:
# get indices with just zeros
idx_delete = []
for i in range(5000,10000):
    if arr_ortho_256[i].sum() == 0 or arr_ground_truth_256[i].sum() == 0:
        idx_delete.append(i)

In [40]:
len(idx_delete)

223

In [41]:
# delete images with just zeros
arr_ortho_256 = np.delete(arr_ortho_256, idx_delete, axis=0)
arr_ground_truth_256 = np.delete(arr_ground_truth_256, idx_delete, axis=0)

In [42]:
print(arr_ortho_256.shape)
print(arr_ground_truth_256.shape)

(16989, 256, 256, 4)
(16989, 256, 256, 1)


In [24]:
arr_ortho_256.shape[0]

19766

In [23]:
19766 + 19777

39543

In [29]:
#ds_file_name  = "/home/philipp/Data/edin_dataset/dataset_512_1.h5"
ds_file_name  = "/mnt/sda1/dataset_256_typ.h5"

tile_size = 256

#ortho_shape = (len(arr_ortho), 512, 512, 4)
#dsm_shape = (len(arr_dsm), 512, 512, 1)
ortho_shape = (arr_ortho_256.shape[0], tile_size, tile_size, 4)
gt_shape = (arr_ground_truth_256.shape[0], tile_size, tile_size, 1)

with h5py.File(ds_file_name, 'w') as hf:
    hf.create_dataset('x_ortho', data=arr_ortho_256, shape=ortho_shape, dtype=np.uint8, chunks=(100,tile_size,tile_size,4))
    hf.create_dataset('y_ground_truth', data=arr_ground_truth_256, shape=gt_shape, dtype=np.uint8, chunks=(100,tile_size,tile_size,1))

In [ ]:
## create

In [25]:
ds_file_name  = "/mnt/sda1/dataset_256_typ.h5"

tile_size = arr_ortho_256.shape[1]

hdf5_store = h5py.File(ds_file_name, 'a')
# seting space to shape 200,512,512,4
x_ortho = hdf5_store.create_dataset("x_ortho", (160000,tile_size,tile_size,4), dtype=np.uint8)
x_ortho[:arr_ortho_256.shape[0],:,:,:] = arr_ortho_256

x_ground_truth = hdf5_store.create_dataset("x_ground_truth", (160000,tile_size,tile_size,1), dtype=np.uint8)
x_ground_truth[:arr_ground_truth_256.shape[0],:,:,:] = arr_ground_truth_256

hdf5_store.close()

In [ ]:
# add data

In [43]:
ds_file_name  = "/mnt/sda1/dataset_256_typ.h5"

tile_size = arr_ortho_256.shape[1]

hdf5_store = h5py.File(ds_file_name, 'a')

In [44]:
hdf5_store

<HDF5 file "dataset_256_typ.h5" (mode r+)>

In [45]:
list(hdf5_store.keys())

['x_ground_truth', 'x_ortho']

In [46]:
last_new = last+arr_ortho_256.shape[0]
print('from: '+ str(last) + ' to: ' + str(last_new))

from: 59328 to: 76317


In [27]:
#last = 39543
#last_new = last+arr_ortho_256.shape[0]
#print('from: '+ str(last) + ' to: ' + str(last_new))

from: 39543 to: 59328


In [47]:
# set dataset
x_ortho = hdf5_store['x_ortho']
# assign data 
x_ortho[last:last_new,:,:,:] = arr_ortho_256

# set dataset
x_ground_truth = hdf5_store['x_ground_truth']
# assign data 
x_ground_truth[last:last_new,:,:,:] = arr_ground_truth_256

hdf5_store.close()

In [48]:
last = last_new
del arr_ortho_256
del arr_ground_truth_256